In [1]:
import pandas as pd
import numpy as np
import auto_feature
#dreload(auto_feature)
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston
from sklearn import linear_model
from sklearn.base import clone
boston = load_boston()
#print(boston.data.shape)

In [2]:
base_address = "../Datasets/"
dataSets = []
dataSets_temp = []
data_names = ['servo','abalone','ozone']
for i in range(0,len(data_names)):
    dataSets_temp.append(pd.read_csv(base_address+data_names[i]+'.csv'))
    temp_data = pd.read_csv(base_address+data_names[i]+'.csv')
    temp_data = pd.get_dummies(temp_data)
    temp_output = pd.DataFrame(temp_data['output'])
    temp_data.drop('output',axis=1,inplace=True)
    temp_data = pd.concat([temp_data, temp_output], axis=1)
    dataSets.append(temp_data)
# Boston Data
b_feat = pd.DataFrame(boston.data)
b_feat.columns = ['feat_0','feat_1','feat_2','feat_3','feat_4','feat_5','feat_6','feat_7','feat_8','feat_9','feat_10','feat_11','feat_12']
b_target = pd.DataFrame(boston.target)
b_target.columns = ['output']
b_data = pd.concat([b_feat,b_target],axis=1)
dataSets.append(b_data)

In [3]:
def get_result(model_,data_, frac):
    model = clone(model_)
    output = data_['output']
    data_.drop('output',axis=1,inplace=True)
    X_train, X_test, y_train, y_test = train_test_split(data_, output, test_size=frac)
    model.fit(X_train,y_train)
    test_result = model.predict(X_test)
    return mean_squared_error(y_test,test_result), test_result, model

In [21]:
def get_result_iter(data, N_iterations, frac, models):
    #get_result(FS,temp_data)
    model_list = []
    Result_mses = []
    for i in range(0,len(models)):
        model_iter_results = []
        for j in range(0,N_iterations):
            temp_data = data.copy(deep=True)
            temp_rs, temp_pred, model = get_result(models[i],temp_data, frac)
            #print model.best_params_
            model_iter_results.append(temp_rs)
            model_list.append(model)
        Result_mses.append(sum(model_iter_results)/len(model_iter_results))
    return Result_mses, model_list

In [5]:
Res_all_ds = []
model_f = []
for i in range(0,len(dataSets)):
    t_data = dataSets[i].copy(deep=True)
    N_iterations = 1
    g_frac = 0.10
    #parameters = {'indpb':np.arange(0.1,0.6,0.2).tolist(), 'mutpb':np.arange(0.1,0.7,0.2).tolist(),
                  #'cxpb':np.arange(0.1,0.7,0.2).tolist(),'N_individual':np.arange(4,15,2).tolist(),
                  #'test_frac':np.arange(0.1,1,0.2)}
    parameters = {'indpb':np.arange(0.1,0.6,0.2).tolist(),'N_individual':[5,10,20,25,35,50],
                  'test_frac':np.arange(0.2,1,0.2)}
    FS = auto_feature.Feature_Stacker(ngen=20,cxpb = 0.6, mutpb = 0.4)
    #grid_model = RandomizedSearchCV(FS, parameters, verbose=True, scoring="mean_squared_error",n_iter=100)
    grid_model = GridSearchCV(FS, parameters, verbose=True, scoring="mean_squared_error")
    g_models = []
    g_models.append(grid_model)
    #g_models.append(linear_model.LinearRegression())
    #g_models.append(linear_model.LassoCV(n_alphas=100))
    Result_t, model_list = get_result_iter(t_data, N_iterations, g_frac, g_models)
    Res_all_ds.append(Result_t)
    model_f.append(model_list)
dataSet_names = ['Servo','Abalone','Ozone','Boston-housing']
model_names = ['Feature Stacker-CV']
Result_test_bench = pd.DataFrame(Res_all_ds)
Result_test_bench.columns = model_names
Result_test_bench.set_index([dataSet_names],inplace=True)
Result_test_bench

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    8.2s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:  8.0min
[Parallel(n_jobs=1)]: Done 200 jobs       | elapsed: 89.8min
[Parallel(n_jobs=1)]: Done 216 out of 216 | elapsed: 105.0min finished


Fitting 3 folds for each of 72 candidates, totalling 216 fits
{'indpb': 0.1, 'N_individual': 5, 'test_frac': 0.80000000000000004}
Fitting 3 folds for each of 72 candidates, totalling 216 fits

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    8.7s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:  8.3min
[Parallel(n_jobs=1)]: Done 200 jobs       | elapsed: 89.4min
[Parallel(n_jobs=1)]: Done 216 out of 216 | elapsed: 107.9min finished



{'indpb': 0.30000000000000004, 'N_individual': 10, 'test_frac': 0.80000000000000004}
Fitting 3 folds for each of 72 candidates, totalling 216 fits

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    9.8s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:  8.9min
[Parallel(n_jobs=1)]: Done 200 jobs       | elapsed: 252.3min
[Parallel(n_jobs=1)]: Done 216 out of 216 | elapsed: 265.3min finished



{'indpb': 0.1, 'N_individual': 5, 'test_frac': 0.80000000000000004}
Fitting 3 folds for each of 72 candidates, totalling 216 fits

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    8.1s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:  8.2min
[Parallel(n_jobs=1)]: Done 200 jobs       | elapsed: 83.9min
[Parallel(n_jobs=1)]: Done 216 out of 216 | elapsed: 97.9min finished



{'indpb': 0.5000000000000001, 'N_individual': 5, 'test_frac': 0.80000000000000004}


,Feature Stacker-CV
Servo,3.344036
Abalone,5.342386
Ozone,22.739452
Boston-housing,26.387494


In [ ]:
grid_model.best_params_

In [ ]:
Result_test_bench.set_index([dataSet_names],inplace=True)
Result_test_bench

In [ ]:
# test_frac = 1, n =5
Result_test_bench_bench

In [ ]:
# test_frac = 0.5, n =25
Result_test_bench

In [ ]:
# test_frac = 1, n =25
Result_test_bench

In [ ]:
for i in range(0,len(dataSets)):
    print dataSets[i].shape

In [ ]:
pd.DataFrame(A)

In [ ]:
dataSet_names = ['Servo','Abalone','Ozone','Boston-housing']
model_names = ['Feature Stacker', 'Linear Regression']
Result_test_bench = pd.DataFrame(Res_all_ds)
Result_test_bench.columns = model_names

In [6]:
len(model_f)

4

In [18]:
mod_FS_GT_1 = model_f

In [11]:
import pickle
from sklearn.externals import joblib

In [ ]:
mod_0_s = pickle.dumps(model_f[0][0])
mod_1_s = pickle.dumps(model_f[1][0])
mod_2_s = pickle.dumps(model_f[2][0])
mod_3_s = pickle.dumps(model_f[3][0])

In [ ]:
joblib.dump(model_f[0][0], 'model-0.pkl')
joblib.dump(model_f[1][0], 'model-1.pkl')
joblib.dump(model_f[2][0], 'model-2.pkl')
joblib.dump(model_f[3][0], 'model-3.pkl')

In [31]:
# Trying it on server
plot_res = []
for i in range(10,700,20):
    print i
    t_data_check = dataSets[0].copy(deep=True)
    N_iterations = 10
    g_frac = 0.10
    g_servo = []
    FS1 = auto_feature.Feature_Stacker(ngen=i,cxpb = 0.6, mutpb = 0.4, indpb= 0.1, N_individual=5, test_frac=0.80000000000000004)
    g_servo.append(FS1)
    Result_servo, model_list_servo = get_result_iter(t_data_check, N_iterations, g_frac, g_servo)
    plot_res.append(Result_servo[0])
    Result_servo

10
30
50
70
90
110
130
150
170
190
210
230
250
270
290
310
330
350
370
390
410
430
450
470
490
510
530
550
570
590
610
630
650
670
690


In [29]:
Result_servo[0]

1.4980921370534261